# Testing backend utility functions for course planning

The purpose of this notebook is to test the course planning utilities in the backend api to ensure proper functionality.

- The first cells in this notebook are the relevant functions from the `utils.py` file. 
- The subsequent cells are testing the various functions.

In [2]:
from typing import List
import re

from models import (CompletedCourse)

In [3]:
# Get the db client
import pymongo
import os
import certifi
from dotenv import load_dotenv
load_dotenv()
CONNECTION_STRING = os.getenv("AZURE_COSMOS_CONNECTION_STRING")
db_client = pymongo.MongoClient(CONNECTION_STRING, tlsCAFile=certifi.where())
db = db_client["db"]

/var/folders/b6/zjrzfvq14h12kvt3_h7c4fwr0000gn/T/ipykernel_5968/935469134.py:8: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  db_client = pymongo.MongoClient(CONNECTION_STRING, tlsCAFile=certifi.where())


In [4]:
# Get the degrees
import json
with open('data/programs.json', 'r') as f:
    degrees = json.load(f)

In [8]:
# Course Planning Utility Functions
# ---------------------------------

def is_course_req(req):
    """Return True or False if the requirement is a course requirement and not a higher order requirement"""
    if type(req) != dict:
        return False
    req_list = req.get('reqList')
    if req_list is None or type(req_list) != list or len(req_list) == 0 or type(req_list[0]) != str:
        return False
    return len(re.findall(r"[a-zA-Z]{2,4}\d{3}", req_list[0])) != 0

def minimum_year_satisfied(units, rule):
    """Return True or False if the student satisfies a rule like minimum third-year"""
    if units < 12 and ('second' in rule or 'third' in rule or 'fourth' in rule or 'fifth' in rule):
        return False
    elif units < 26.5 and ('third' in rule or 'fourth' in rule or 'fifth' in rule):
        return False
    elif units < 41.5 and ('fourth' in rule or 'fifth' in rule):
        return False
    elif units > 42 and 'fifth' in rule:
        return False
    return True

def is_req_satisfied(req, units, completed_course_codes):
    """Return True or False if the requirement is satisfied given completed courses"""
    if type(req) == str: # min year standing, could be other, may need to look into this...
        return minimum_year_satisfied(units, req)
    
    if type(req) == list:
        return all(is_req_satisfied(r, units, completed_course_codes) for r in req)

    qty = req.get('quantity')
    reqlist = req.get('reqList')

    if is_course_req(req): 
        if qty == "ALL":
            if not all(code in completed_course_codes for code in req['reqList']):
                return False
        elif type(qty) == int:
            ct = 0
            for i in range(len(reqlist)):
                code = reqlist[i]
                if code in completed_course_codes:
                    ct += 1
            if ct < qty:
                return False
        elif qty is None:
            # print("course req, qty is none")
            # pprint(req)
            return False
        
    else: # higher order prereqs
        if qty == "ALL":
            return all(is_req_satisfied(x, units, completed_course_codes) for x in reqlist)
        elif type(qty) == int:
            ct = 0
            for i in range(len(reqlist)):
                if is_req_satisfied(reqlist[i], units, completed_course_codes):
                    ct += 1
            if ct < qty:
                return False
        elif qty is None:
            # print("upper req, qty is none")
            # print(req)
            return False
    
    return True

def can_take(course, units, completed_course_codes):
    """Return True or False if all requirements are satisfied for course given completed courses"""
    prereqs = course.get('prerequisites')
    if not prereqs:
        return True
    return all(is_req_satisfied(prereq, units, completed_course_codes) for prereq in prereqs)

def get_unsatisfied_course_reqs(req, units, completed_course_codes):
    new = []
    if is_course_req(req):
        if not is_req_satisfied(req, units, completed_course_codes):
            return [req]
    elif type(req) == list:
        for r in req:
            if not is_req_satisfied(req, units, completed_course_codes):
                new += get_unsatisfied_course_reqs(r, units, completed_course_codes)
    elif type(req) == str: # other reqs... dont bother with them here
        pass
    else:
        if not is_req_satisfied(req, units, completed_course_codes):
            new += get_unsatisfied_course_reqs(req['reqList'], units, completed_course_codes)
    return new

def get_course_reqs_left(degree, units, completed_course_codes):
    """Return a dictionary of the course requirements left. Keys are year and values are a list of course reqs."""
    reqs_left = {}

    degree_requirements = degree.get('requirements')
    if degree_requirements is None:
        return {}
    
    # Get unsatisfied reqs
    for info, req in degree_requirements.items():
        reqs_left[info] = get_unsatisfied_course_reqs(req, units, completed_course_codes)

    # Modify based on what's needed to be taken
    for info, reqs in reqs_left.items():
        for req in reqs:
            qty = req.get('quantity')
            if qty is None: # Unfortunately there are some unparsed requirements.
                continue
            if qty == 'ALL':
                req['reqList'] = [code for code in req['reqList'] if code not in completed_course_codes]

            elif type(qty) == int:
                ct = 0
                new_codes = []
                for code in req['reqList']:
                    if code in completed_course_codes:
                        ct += 1
                    else:
                        new_codes.append(code)
                req['quantity'] = qty - ct
                req['reqList'] = new_codes
            else:
                raise Exception

    reqs_left = {k:v for k,v in reqs_left.items() if v!=[]}
    return reqs_left

def get_courses_left(degree, units, completed_course_codes):
    course_reqs_left = get_course_reqs_left(degree, units, completed_course_codes)
    course_reqs = []
    for reqs in course_reqs_left.values():
        for req in reqs:
            course_reqs.append(req)
    course_codes = []
    for req in course_reqs:
        if any(type(x) == dict for x in req['reqList']):
            continue
        course_codes += req['reqList']
    
    return course_codes

def get_useful_courses(db, degree, units, completed_course_codes):
    """Returns a triplet containing courses left in degree which (potential_courses, missing_prereq, not_offered)"""
    courses_left = get_courses_left(degree, units, completed_course_codes)
    courses = db.courses.aggregate([{'$match': {'_id': {'$in': courses_left}}}])
    potential_courses = [x['_id'] for x in courses if can_take(x, units, completed_course_codes)]
    missing_prereq = [x['_id'] for x in courses if x not in potential_courses]
    not_offered = [code for code in courses_left if code not in potential_courses]
    return (potential_courses, missing_prereq, not_offered)

## Test case 1

|Degree|Degree code|Completed courses
|---|---|---|
| Theatre History | BFA-THFM | THEA105, THEA111, THEA120A, THEA132A, THEA236, ATWP135

In [5]:
degree_code = 'BSC-CTSC'
degree = degrees[degree_code]
completed_course_codes = ['MATH109', 'PHIL201', 'THEA150', 'CSC110', 'ATWP135']
units = 7.5

In [7]:
# degree['requirements']

In [13]:
get_course_reqs_left(degree, units, completed_course_codes)

{'Year 1': [{'quantity': 'ALL',
   'reqList': ['THEA103',
    'THEA104',
    'THEA105',
    'THEA106',
    'THEA120A',
    'THEA206']}]}